In [28]:
import pandas as pd
import numpy as np
import csv
import pickle
import seaborn as sns
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import matplotlib.pyplot as plt

In [29]:
Path='/home/mgander/Atlantic/data/Viability/'
M = pd.read_pickle(f'{Path}/M_all_all.pkl')
M = M[M.Source!='PRISM1'].copy()
M['log_dose']=np.log10(M['Dose'])
M['key']=M['Cello']+'_'+M['PubChem_CID']
M['log_dose_rounded']= (M['log_dose']*10).astype('int')
sources = sorted(set(M['Source']))
sources

['CTD2', 'CTPR', 'DTP', 'GDSC1', 'GDSC2', 'MR_NCI60', 'PRISM2']

In [30]:
def fct(x, a, b, c, d):
    return a + (b-a) / (1+10**(-c*(x-d)))
def add_ccl_and_drugs_annot(df):
    df['Cello']=[a.split('_')[0] for a in df.index]
    df['PubChem_CID']=[a.split('_')[1] for a in df.index]
    df['pEC50'] = - df['-pec50']
    del df['-pec50']
    return(df)
def get_intervals(dfg):
    # rounded dose onto 1 decimal
    lefts = [a[1]['log_dose_rounded'].min() for a in list(dfg.groupby('Cello'))]
    rights = [a[1]['log_dose_rounded'].max() for a in list(dfg.groupby('Cello'))]

    left_maj = list(pd.DataFrame(lefts, columns=['v']).value_counts().index)[0][0]
    right_maj = list(pd.DataFrame(rights, columns=['v']).value_counts().index)[0][0]
    return(left_maj, right_maj)

In [31]:
F={}
F['CTD2'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_ctd2.pkl'))
F['CTPR'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_ctrp.pkl'))
F['DTP'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_dtp.pkl'))
F['GDSC1'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_gdsc1.pkl'))
F['GDSC2'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_gdsc2.pkl'))
F['MR_NCI60'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_inhouse.pkl'))
F['PRISM2'] = add_ccl_and_drugs_annot(pd.read_pickle(f'{Path}/dff_prism.pkl'))

In [32]:
def get_sensitivities_for_Mss(Mss, Fs, cut0=0.3, cut1=0.6):
    Dm={}
    
    keys = sorted(set(Mss['key']))
    pec50s = Fs.loc[keys]['pEC50']
    nans = pec50s[~np.isfinite(pec50s)].index
    pec50s=pec50s[np.isfinite(pec50s)]
    x0,x1=np.array(get_intervals(Mss))/10
    
    pec50s[pec50s>x1]=x1
    pec50s[pec50s<x0]=x0
    
    med = np.median(pec50s)
    cutoff0 = med-cut0
    cutoff1 = med-cut1
    
    inds=list(pec50s.index)
    vals=list(pec50s.values)
    for i in range(len(inds)):
        v=vals[i]
        if v>cutoff0:
            Dm[inds[i]]='not'
        elif v>cutoff1:
            Dm[inds[i]]='maybe'
        else:
            Dm[inds[i]]='yes'
    for ind in nans:
        Dm[ind]='maybe'
    return(Dm, pec50s)

In [33]:
for i in range(len(sources)):
    Dmt={}
    source = sources[i]
    Fs=F[source]
    Ms=M[M['Source']==source].copy()
    drugs = sorted(set(Fs['PubChem_CID']))
    for dr in tqdm(drugs):
        Mss = Ms[Ms['PubChem_CID']==dr].copy()
        if len(Mss)>0:
            Dm, pec50s = get_sensitivities_for_Mss(Mss, Fs)
            for k in Dm.keys():
                Dmt[k]=Dm[k]
    F[source]['sensitive']=F[source].index.map(Dmt)

 57%|███████████████████████████████████████████████▎                                   | 13047/22886 [7:22:52<5:34:21,  2.04s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 86%|██████████████████████████████████████████████████████████████████████▊           | 19760/22886 [11:09:43<1:43:23,  1.98s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [34]:
F[sources[0]].value_counts('sensitive')

sensitive
not      304249
yes       50075
maybe     31680
Name: count, dtype: int64

In [35]:
import pickle
with open('F_annot.pkl', 'wb') as file:
    pickle.dump(F, file)

In [ ]:
import pickle
with open('F_annot.pkl', 'wb') as file:
    pickle.dump(F, file)

import pickle
# Load from pickle file
with open('F_annot.pkl', 'rb') as file:
    F = pickle.load(file)

In [38]:
Df ={}
for source in sources:
    Df[source]={}
    Fs = F[source]
    ccls = [a.split('_')[0] for a in Fs.index]
    drs = [a.split('_')[1] for a in Fs.index]
    Fs['PubChem_CID']=drs
    Fs['Cello']=ccls
    Fs['PubChem_CID'] = Fs['PubChem_CID'].astype('category')
    Fs['Cello'] = Fs['Cello'].astype('category')
    for dr in tqdm(sorted(set(drs))):
        Fss = Fs[Fs['PubChem_CID']==dr].copy()
        Df[source][dr]={}
        ccls = list(Fss['Cello'])
        sens = list(Fss['sensitive'])
        for i in range(len(ccls)):
            Df[source][dr][ccls[i]]=sens[i]

100%|████████████████████████████████████████████████████████████████████████████████████████| 1441/1441 [00:02<00:00, 602.16it/s]


In [39]:
import pickle
with open('Sensitivity_dict.pkl', 'wb') as file:
    pickle.dump(Df, file)

In [ ]:
import pickle
with open('Sensitivity_dict.pkl', 'wb') as file:
    pickle.dump(Df, file)

import pickle
# Load from pickle file
with open('Sensitivity_dict.pkl', 'rb') as file:
    Df = pickle.load(file)

In [27]:
sorted(set(F['GDSC1']['PubChem_CID']))

['10027278',
 '10074640',
 '10077147',
 '10109823',
 '10113978',
 '10127622',
 '10172943',
 '10184653',
 '10302451',
 '10341154',
 '10368812',
 '10384072',
 '10459196',
 '10461815',
 '104741',
 '104842',
 '107970',
 '11152667',
 '11175137',
 '11178236',
 '11234052',
 '11282283',
 '11327430',
 '11338033',
 '11364421',
 '11373846',
 '11455910',
 '11467730',
 '11485656',
 '11488320',
 '11493598',
 '11511120',
 '11609586',
 '11617559',
 '11624601',
 '11625818',
 '11626560',
 '11626927',
 '11634725',
 '11640390',
 '11647372',
 '11667893',
 '11676786',
 '11707110',
 '11713159',
 '11754511',
 '11754711',
 '11784504',
 '117918970',
 '117996541',
 '11844351',
 '11960529',
 '11977753',
 '123631',
 '126565',
 '126941',
 '135398501',
 '135398516',
 '135400873',
 '135410875',
 '135418940',
 '135421197',
 '135423438',
 '135440466',
 '135465539',
 '135499520',
 '135539077',
 '135564599',
 '135565082',
 '135659062',
 '1401',
 '145945963',
 '148124',
 '14998470',
 '156422',
 '159324',
 '15953870',
 '16